In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local").appName("LinearRegression").getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [12]:
#loading the training data
data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [14]:
data.show() #actual format spark needs to run a machine learning model

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [22]:
data.head(1)[0]['features']

SparseVector(10, {0: 0.4551, 1: 0.3664, 2: -0.3826, 3: -0.4458, 4: 0.3311, 5: 0.8067, 6: -0.2624, 7: -0.4485, 8: -0.0727, 9: 0.5658})

In [23]:
lr = LinearRegression(featuresCol="features",labelCol="label",predictionCol="prediction")

In [24]:
lrModel = lr.fit(data)

In [28]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [29]:
lrModel.intercept

0.14228558260358093

In [30]:
training_summary = lrModel.summary

In [31]:
training_summary.r2

0.027839179518600154

In [32]:
training_summary.rootMeanSquaredError

10.16309157133015

# Splitting the data in train and test

In [47]:
train_data, test_data = data.randomSplit([0.7,0.3])

In [48]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                358|
|   mean|0.39207737965774025|
| stddev|  9.973372263036056|
|    min|-26.805483428483072|
|    max|  27.78383192005107|
+-------+-------------------+



In [49]:
test_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 143|
|   mean|-0.08155525872573288|
| stddev|  11.163418538619469|
|    min| -28.571478869743427|
|    max|  26.903524792043335|
+-------+--------------------+



In [50]:
split_model = lr.fit(train_data)

In [51]:
testResults = split_model.evaluate(test_data)

In [53]:
testResults.rootMeanSquaredError

11.238521353262731

# Prediction

In [54]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [56]:
split_model.transform(unlabeled_data).show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -0.3702295538262638|
|(10,[0,1,2,3,4,5,...| -1.4244974210699999|
|(10,[0,1,2,3,4,5,...|-0.14782089718287467|
|(10,[0,1,2,3,4,5,...|  2.2114295114352296|
|(10,[0,1,2,3,4,5,...|  1.5895692684296086|
|(10,[0,1,2,3,4,5,...|  -3.638668980117302|
|(10,[0,1,2,3,4,5,...|   1.571867051520616|
|(10,[0,1,2,3,4,5,...|  2.8723416526302517|
|(10,[0,1,2,3,4,5,...|  1.8001355692770105|
|(10,[0,1,2,3,4,5,...|  1.4414554522722893|
|(10,[0,1,2,3,4,5,...|  0.7028591888228359|
|(10,[0,1,2,3,4,5,...|  2.0454897502364706|
|(10,[0,1,2,3,4,5,...| -1.3582742484564283|
|(10,[0,1,2,3,4,5,...|  2.7598199756377015|
|(10,[0,1,2,3,4,5,...|  0.6679963709392468|
|(10,[0,1,2,3,4,5,...|   4.786095546885312|
|(10,[0,1,2,3,4,5,...|  -4.231588012825986|
|(10,[0,1,2,3,4,5,...| -1.1361208241538006|
|(10,[0,1,2,3,4,5,...|  1.0544209108145692|
|(10,[0,1,2,3,4,5,...|   3.57332